# 0. 들어가기 전에
 - device : Cuda를 사용중이면 현재 device는 Cuda이고, 아니면 현재 device는 CPU 이다.
 - to(device) : 이 명령을 작성해야 하는 이유는, Cuda에서 연산을 진행할 때 Torch tensor가 아닌 Torch Cuda tensor를 이용해야 하기 때문에 이 명령어를 작성한다.
 - print(device) : 현재 사용중인 device 확인 가능!
 - print(torch.cuda.is_available()) : 현재 Cuda 사용 가능한지 확인 가능!

# 1. CNN의 학습 단계
 - 라이브러리 로드
 - GPU 사용 설정 및 random value를 위한 seed 설정
 - 하이퍼파라미터 설정(LR, epoch, batch_size 등)
 - 데이터셋 로드 및 학습에 편하게 Loader로 만들기
 - CNN 모델 만들어 두기
 - Loss와 Optimizer 선택
 - CNN 모델 학습 및 loss 확인
 - 모델 평가

# 2. 우리가 실습할 CNN 구조
### (1) Input
 - MNIST의 이미지 데이터

<br>

### (2) CNN
 #### 1) Layer 1
  - Convolution
  - ReLU
  - MaxPool
  
 #### 2) Layer 2
  - Convolution
  - ReLU
  - MaxPool
  
 #### 3) FC Layer(Fully Connected Layer)
  - view : FC를 위해 Flatten을 시켜 직선으로 쫙 펼치는 과정
  - FC

<br>

### (3) Cross Entropy Loss
 - SoftMax
 - NLL loss

# 3. MNIST CNN 실습

In [1]:
# 1. 라이브러리 로드
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
# 2. GPU 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu' # cuda가 사용 가능한 상황이면 cuda를 쓰고, 아니면 CPU를 사용

torch.manual_seed(777) # random value 고정!
if device == 'cuda': # cuda일 때
    torch.cuda.manual_seed_all(777) # random value 고정!

In [3]:
# 3. 하이퍼 파라미터 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
# 4. 데이터 셋 가져오기
mnist_train = dsets.MNIST(root = 'MNIST_data/', train = True, transform = transforms.ToTensor(), download = True) # train = True를 설정해서 트레인용 데이터를 받아아고, transform으로 input 데이터를 tensor화 하고, download로 다운 받아서 이용
mnist_test = dsets.MNIST(root = 'MNIST_data/', train = False, transform = transforms.ToTensor(), download = True) # train = False를 설정해서 테스트용 데이터를 받아온다. 

In [5]:
# 5. 데이터 로더
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, batch_size = batch_size, shuffle = True, drop_last = True) # 데이터 셋, 배치 사이즈, 데이터 섞는 여부, drop last 설정

In [6]:
# 6. CNN 모델 만들기
class CNN(torch.nn.Module): # Pytorch의 모듈을 상속받아 CNN 클래스를 설정한다.
    def __init__(self):
        super(CNN, self).__init__() # 주의! 필수 작업!

        # Layer 1
        self.layer1 = torch.nn.Sequential( # Sequential로 레이어를 쌓을 수 있다.
        torch.nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1), # Convolution
        torch.nn.ReLU(), # ReLU
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2)) # MaxPool
    
        # Layer 2
        self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2))
      
        # FC Layer
        self.fc = torch.nn.Linear(7*7*64, 10, bias = True) # 앞 과정을 거친 size를 미리 계산해 보면 64 채널의 7 * 7 크기가 나온다. 그걸 FC를 통해 Flatten으로 직선으로 쫙 펼치게 된다. FC의 output은 10개(0 ~ 9) 이다.
        torch.nn.init.xavier_uniform_(self.fc.weight) # FC Layer의 weight를 xavier로 초기화
   
    # 모델의 forward 진행 과정
    def forward(self, x):
        out = self.layer1(x) # Layer 1 통과
        out = self.layer2(out) # Layer 2 통과
        out = out.view(out.size(0), -1) # FC Layer을 위해 Flatten으로 한 줄(-1)으로 펼치기.
        out = self.fc(out) # FC Layer 통과
        return out
    
# 모델 설정
model = CNN().to(device)
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)

In [7]:
# 7. Loss와 Optimizer 설정
criterion = torch.nn.CrossEntropyLoss().to(device) # Cross Entropy Loss 사용
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # Adam Optimizer 사용

total_batch = len(data_loader) # batch 몇개 인지 확인
print('Learning started. It takes sometime.')

Learning started. It takes sometime.


In [8]:
# 8. 학습 시작!
for epoch in range(training_epochs):
    avg_cost = 0
  
    for X, Y in data_loader:
        X = X.to(device) # 학습할 이미지
        Y = Y.to(device) # 학습할 라벨

        hypothesis = model(X) # 모델 이용!
        cost = criterion(hypothesis, Y) # cost 계산

        # BP
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
        avg_cost += cost / total_batch
    
    print('[Epoch: {:4} cost = {:9}'.format(epoch + 1, avg_cost))
    
print('Learning Finished!')

[Epoch:    1 cost = 0.22395089268684387
[Epoch:    2 cost = 0.06211347132921219
[Epoch:    3 cost = 0.04482545703649521
[Epoch:    4 cost = 0.035523492842912674
[Epoch:    5 cost = 0.029017852619290352
[Epoch:    6 cost = 0.024802125990390778
[Epoch:    7 cost = 0.020738985389471054
[Epoch:    8 cost = 0.01811862364411354
[Epoch:    9 cost = 0.015076704323291779
[Epoch:   10 cost = 0.012611476704478264
[Epoch:   11 cost = 0.01029976923018694
[Epoch:   12 cost = 0.010091177187860012
[Epoch:   13 cost = 0.008734307251870632
[Epoch:   14 cost = 0.007107528392225504
[Epoch:   15 cost = 0.00541576137766242
Learning Finished!


In [9]:
# 9. 모델 평가
with torch.no_grad(): # Test할 때는 학습을 안함! No gradient
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device) # X_test 값을 불러와서 모델에 한 번에 집어넣으려고 쫙 펼침
    Y_test = mnist_test.test_labels.to(device)
  
    prediction = model(X_test) # 모델에 X_test를 한번에 다 집어넣음! (No batch)
  
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item()) # 98.5 %

Accuracy: 0.9871999621391296


/home/cloudera/.local/lib/python3.6/site-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/home/cloudera/.local/lib/python3.6/site-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


# 4. 더 생각해보기
 - 레이어를 더 깊게 쌓으면 어떻게 될까? 정확도가 더 높아지지 않을까? 레이어를 더 깊게 쌓아보자!
### (1) Input
 - MNIST의 이미지 데이터

<br>

### (2) CNN
 #### 1) Layer 1
  - Convolution
  - ReLU
  - MaxPool
  
 #### 2) Layer 2
  - Convolution
  - ReLU
  - MaxPool

 #### 3) Layer 3
  - Convolution
  - ReLU
  - MaxPool

 #### 4) FC Layer 1 (Fully Connected Layer)
  - view : FC를 위해 Flatten을 시켜 직선으로 쫙 펼치는 과정
  - FC 1

 #### 5) FC Layer 2 (Fully Connected Layer)
  - ReLU
  - FC 2

<br>

### (3) Cross Entropy Loss
 - SoftMax
 - NLL loss

# 5. MNIST 더 깊은 CNN 실습

```python
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    # Layer 1
    self.layer1 = torch.nn.Sequential(
      torch.nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
      torch.nn.ReLU(),
      torch.nn.MaxPool2d(kernel_size = 2, stride = 2))

    # Layer 2
    self.layer2 = torch.nn.Sequential(
      torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
      torch.nn.ReLU(),
      torch.nn.MaxPool2d(kernel_size = 2, stride = 2))

    # Layer 3
    self.layer3 = torch.nn.Sequential(
      torch.nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
      torch.nn.ReLU(),
      torch.nn.MaxPool2d(Kernel_size = 2, stride = 2))

    # FC Layer 1
    self.fc1 = torch.nn.Linear(3 * 3 * 128, 625)
    self.relu = nn.ReLU()

    # FC Layer 2
    self.fc2 = torch.nn.Linear(625, 10, bias = True)

    torch.nn.init.xavier_uniform_(self. fc1.weight)
    torch.nn.init.xavier_uniform_(self. fc2.weight)
    
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    
    out = out.view(out.size(0), -1)
    
    out = self.fc1(out)
    out = self.relu(out)
    out = self.fc2(out)
    return out

# 6. MNIST 더 깊은 CNN 실습 결과
 - Accuracy가 0.97이 나왔다. 더 깊게 쌓았는데 정확도는 더 낮아졌다!
 - 결론 : 모델을 깊게 쌓는 것도 중요하지만, 모델의 아키텍쳐를 공부하며 효율적으로 쌓는 것이 더 중요하다.

# 7. 꿀팁
 - 모델을 만들고서 학습시키기 전에, 모델이 오류없이 잘 만들어 졌나 확인하는 절차가 있으면 좋다!
 - 모델의 각 레이어의 인풋, 아웃풋 값을 잘 설정해야 오류가 없다. Convolution을 하고 나서, Pool을 하고 나서의 Output을 잘 계산해서 다음 레이어의 인풋 값 등을 잘 설정해줘야 오류가 없다.

```python
model = CNN().to(device)
value = torch.Tensor(1, 1, 28, 28).to(device)
print( (model(value)).shape )
```